# More Realistic Simulation

This is the difference between a "tutorial follower" and a "developer." To make this project get you hired in Hong Kong, you need to add **Friction** and **Complexity**.

Here are three specific tiers of upgrades you can build on top of your current script.

### Tier 1: The "Realism" Upgrade (Transaction Costs)
**The Problem:** Your current script assumes trading is free. In reality, every trade costs money (commission + slippage). A strategy that makes 10% profit with 1,000 trades might actually lose money once you pay 0.1% per trade.

**The Task:**
1.  Define a cost variable (e.g., `COST = 0.001` for 10bps).
2.  Detect *when* a trade happens. A trade happens when the signal *changes* (e.g., from 0 to 1, or 1 to 0).
3.  Subtract the cost from your returns on those specific days.

**The Code Add-On:**
```python
# Calculate when a trade occurs (Buy or Sell)
# .diff() calculates the difference between row N and row N-1
# If signal goes 0 -> 1, diff is 1. If 1 -> 0, diff is -1.
# We take the absolute value because we pay fees on BOTH buying and selling.
df['Trades'] = df['Signal'].diff().abs()

# Subtract costs from the strategy return
# We shift the cost to occur on the day of execution (same as the return)
# Note: We fillNA(0) because the first row can't differ from a previous row
transaction_cost = 0.001  # 0.1% per trade
df['Strategy_Net_Returns'] = df['Strategy_Returns'] - (df['Trades'].shift(1) * transaction_cost)

# Re-calculate your cumulative wealth with the Net Returns
df['Net_Equity'] = (1 + df['Strategy_Net_Returns']).cumprod()
```

### Tier 2: The "Engineer" Upgrade (Parameter Optimization)
**The Problem:** Why did you pick 50 and 200 days? Because a blog told you to?
A Quant tests *everything*. Maybe 40 and 180 is better for NVIDIA?

**The Task:**
Write a loop that tests every combination of "Short Window" (10 to 50) and "Long Window" (100 to 250) and tells you which pair had the highest Sharpe Ratio.

**The Code Structure:**
```python
best_sharpe = 0
best_params = (0, 0)

# Nested loops to test combinations
for short_window in range(20, 60, 5): # Test 20, 25, 30...
    for long_window in range(150, 250, 10): # Test 150, 160...
        
        # 1. Calculate SMAs with these specific windows
        # 2. Generate Signals
        # 3. Calculate Sharpe
        
        if current_sharpe > best_sharpe:
            best_sharpe = current_sharpe
            best_params = (short_window, long_window)

print(f"Optimal Strategy: Buy at {best_params[0]} / {best_params[1]}")
```
*Warning:* If you put this on your resume, be prepared to discuss "Overfitting." (Just because 42/186 worked in the past doesn't mean it will work in the future).

### Tier 3: The "Product" Upgrade (Interactive Dashboard)
**The Problem:** A Jupyter Notebook is ugly. You can't send a notebook to a Recruiter.
**The Fix:** Turn your script into a web app using **Streamlit**. It requires no HTML/CSS knowledge, just Python.

**The Task:**
1.  Install streamlit (`pip install streamlit`).
2.  Create a file `app.py`.
3.  Add sliders so the user can change the moving averages themselves.

**The Code Snippet:**
```python
import streamlit as st

st.title("HK Quant Backtester")

# Sidebar for user inputs
ticker = st.sidebar.text_input("Ticker Symbol", "NVDA")
short_w = st.sidebar.slider("Short Window", 10, 100, 50)
long_w = st.sidebar.slider("Long Window", 100, 300, 200)

# ... [Insert your Backtest Logic Here using these variables] ...

# Display results
st.line_chart(df[['Strategy_Equity', 'Buy_Hold_Equity']])
st.write(f"Sharpe Ratio: {sharpe:.2f}")
```
**The ROI:** You can host this for free on "Streamlit Cloud" and put the link in your LinkedIn bio. "Check out my algorithmic trading engine" sounds much better than "I know Python."

### Which one should you do first?
Do **Tier 1 (Transaction Costs)** immediately. It shows you understand that trading isn't free. This is the #1 thing that separates academic projects from industry-ready code.

# Other Trading Strategies

To move beyond simple RSI strategies and prove you are "Quant Material," you need to demonstrate you understand **Market Structure** (Volatility) and **Statistical Relationships** (Correlation/Cointegration).

Here are three distinct strategy types that act as excellent portfolio projects. They use the same `yfinance` + `pandas` stack but test completely different skills.

---

### Strategy 1: The "Volatility Breakout" (Momentum)
*   **The Logic:** Markets alternate between "Quiet" (Low Volatility) and "Explosive" (High Volatility). This strategy doesn't care about the direction; it bets that a quiet period will be followed by a big move.
*   **Why it gets you hired:** It shows you understand **Risk-Adjusted Sizing** (using ATR) rather than just price.

**The Code Concept:**
1.  Calculate the **ATR** (Average True Range) to measure volatility.
2.  Identify a "squeeze": When the current ATR is lower than the 30-day average ATR (the calm before the storm).
3.  **Signal:** If Price breaks the *Highest High* of the last 20 days + 1 ATR, **BUY**.
4.  **Exit:** If Price falls below the *Lowest Low* of the last 10 days, **SELL**.

### Strategy 2: "Pairs Trading" (Statistical Arbitrage)
*   **The Logic:** You find two stocks that are historically "tied" together (e.g., Coke & Pepsi, or Gold & Gold Miners). When they drift apart, you Short the winner and Long the loser, betting they will snap back like a rubber band.
*   **Why it gets you hired:** This is a true "Hedge Fund" strategy. It is **Market Neutral** (you don't care if the market goes up or down, only that the *relationship* holds).

**The Code Concept:**
You need `statsmodels` for this one.

```python
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint

# 1. Get Data for a Correlated Pair
# GLD (Gold ETF) and GDX (Gold Miners ETF) are a classic pair
data = yf.download(['GLD', 'GDX'], start='2020-01-01')['Adj Close']

# 2. Test for Cointegration (The "Rubber Band" Test)
# P-value < 0.05 means they are statistically linked
score, pvalue, _ = coint(data['GLD'], data['GDX'])
print(f"Cointegration P-Value: {pvalue}")

# 3. Calculate the Spread (The Trading Signal)
# We find the Hedge Ratio to make them comparable
import statsmodels.api as sm
model = sm.OLS(data['GLD'], data['GDX']).fit()
hedge_ratio = model.params[0]

# The Spread is the difference between the two, adjusted by the ratio
data['Spread'] = data['GLD'] - (hedge_ratio * data['GDX'])

# 4. Generate Z-Score (How far is the rubber band stretched?)
mean_spread = data['Spread'].rolling(window=30).mean()
std_spread = data['Spread'].rolling(window=30).std()
data['Z_Score'] = (data['Spread'] - mean_spread) / std_spread

# 5. Signal: Buy when Z-Score < -2, Sell when Z-Score > 2
data['Signal'] = np.where(data['Z_Score'] < -2, 1, np.where(data['Z_Score'] > 2, -1, 0))
```

### Strategy 3: VWAP Mean Reversion (Institutional Execution)
*   **The Logic:** **VWAP** (Volume Weighted Average Price) is the benchmark used by institutions. If the price is significantly below VWAP, it is considered "cheap" by big banks. If above, it is "expensive."
*   **Why it gets you hired:** It shows you understand **Liquidity** and **Volume**, not just Price.

**The Code Concept:**
VWAP is not a standard column; you must build it.

```python
# Assuming you downloaded 'High', 'Low', 'Close', 'Volume'
df = yf.download('NVDA', start='2024-01-01', interval='1h') # Intraday data works best

# 1. Calculate Typical Price
df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3

# 2. Calculate VWAP (Cumulative Price * Volume / Cumulative Volume)
# Note: Real VWAP resets every day. For a project, you can use a rolling window.
df['VWAP'] = (df['Typical_Price'] * df['Volume']).rolling(window=24).sum() / df['Volume'].rolling(window=24).sum()

# 3. Signal
# Buy if Price crosses ABOVE VWAP (Bullish confirmation)
# OR Buy if Price is 2 Standard Deviations BELOW VWAP (Mean Reversion)
```

### Recommendation for Your GitHub
Add **Strategy 2 (Pairs Trading)** to your repository next.
1.  It forces you to use a new library (`statsmodels`).
2.  It introduces a statistical concept (Cointegration vs. Correlation).
3.  It visually looks impressive when you plot the "Spread" snapping back to zero.

# Fun Stuff

Since you are currently in **Singapore** (or your system is routed there), and your career goal is **Hong Kong**, let's bridge the gap. "Fun" in the Quant world usually means **Visualizations**, **Gambling Mathematics**, or **Alternative Data**.

Here are three "Edutainment" projects that are technically impressive but actually fun to build and watch.

### 1. The "Monte Carlo" Casino (Simulating the Future)
Instead of backtesting the past, simulate 1,000 possible futures. This is visually mesmerizing and is the foundation of **Value at Risk (VaR)** models used by every bank in Singapore and HK.

*   **The Concept:** "If I invest $10,000 in Bitcoin today, what are the 1,000 different ways this could play out over the next year?"
*   **The Fun Part:** You generate a "Spaghetti Plot"—a chaotic, beautiful mess of lines showing best-case (Lambo) and worst-case (Zero) scenarios.

**The Code Snippet (Matplotlib Magic):**
```python
import numpy as np
import matplotlib.pyplot as plt

# Setup: 1 year, 252 trading days, 1000 simulations
days = 252
simulations = 1000
start_price = 100

# Assumptions: 10% expected return, 20% volatility (Change these!)
mu = 0.10 / days
sigma = 0.20 / np.sqrt(days)

# The Math: Geometric Brownian Motion (The standard model for stocks)
# We generate random daily shocks
shocks = np.random.normal(mu, sigma, (days, simulations))
price_paths = start_price * np.cumprod(np.exp(shocks), axis=0)

# Visualizing the Chaos
plt.figure(figsize=(10,6))
plt.plot(price_paths[:, :50]) # Plot first 50 paths to keep it clean
plt.title(f'Monte Carlo Simulation: {simulations} Possible Futures')
plt.xlabel('Days')
plt.ylabel('Price ($)')
plt.show()
```
*   **ROI Factor:** This proves you understand **Stochastic Calculus** without needing to write the differential equations on a whiteboard.

### 2. The "Hype Hunter" (NLP & Sentiment Analysis)
Finance isn't just numbers; it's psychology.
*   **The Project:** Build a script that scrapes headlines or Reddit (r/WallStreetBets) and correlates "Hype" with "Price."
*   **The Fun Part:** You get to analyze text data. Does the word "Moon" appearing 500 times actually predict a price jump?
*   **Tools:**
    *   `Library: TextBlob` or `VADER` (for sentiment scores: Positive/Negative).
    *   `Library: PRAW` (to scrape Reddit) or `yfinance` news.

**Simple Idea:**
1.  Download news headlines for Tesla.
2.  Score them: +1 (Positive), -1 (Negative).
3.  Plot the "Sentiment Score" line against the "Stock Price" line.
4.  *Spoiler:* You will often find they move together!

### 3. The "Singapore vs. Hong Kong" Showdown (Correlation Heatmaps)
Since you are physically in **Singapore** but professionally aimed at **Hong Kong**, analyze the relationship between the two financial hubs.

*   **The Project:** A "Macro-Correlation" Heatmap.
*   **The Data:**
    *   **STI (Straits Times Index):** The Singapore Benchmark (`^STI`).
    *   **HSI (Hang Seng Index):** The Hong Kong Benchmark (`^HSI`).
    *   Add: S&P 500 (`SPY`) and Gold (`GLD`).
*   **The Fun Part:** Use `seaborn` to create a colored heatmap.
    *   *Question:* When HK crashes, does Singapore crash too? or does money flow from HK to SG? (This is a huge talking point in finance right now).

**The "Pretty Chart" Code:**
```python
import seaborn as sns

tickers = ['^STI', '^HSI', 'SPY', 'GLD']
data = yf.download(tickers, start='2020-01-01')['Adj Close']

# Calculate daily returns
corr_matrix = data.pct_change().corr()

# Plot the Heatmap
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Singapore vs. HK vs. USA: Who Moves Together?')
plt.show()
```

### Summary of "Fun"
1.  **Monte Carlo:** If you like chaos and gambling math.
2.  **NLP/Sentiment:** If you like psychology and social media.
3.  **Correlation Heatmaps:** If you want to understand the macro-economics of your current location (SG) vs. your target location (HK).

**My Advice:** Do the **Monte Carlo** one first. It produces the coolest looking chart to show your friends (or a recruiter), and it makes you feel like a wizard predicting the future.